In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.svm import SVC
from sklearn import svm

In [9]:
DATA_PATH = '/content/drive/MyDrive/MlProject1/ML_project1/'
westEth=[]
labelwe=[]
hashList=[]
count=2
for folder in os.listdir(DATA_PATH):
    if folder=="Gender":
      continue;
    print(">>>Reading ",folder)
    count-=1
    print(count)
    
    for file in os.listdir(DATA_PATH+folder):
         if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
              westEth.append(resize(img, (156, 156, 3)))
              hashList.append(hsh)
              labelwe.append(count)
westEth=np.array(westEth)
labelwe=np.array(labelwe)

>>>Reading  western
1
>>>Reading  ethenic
0


In [17]:
resultPath = '/content/drive/MyDrive/Results1'
train_folder = os.listdir(DATA_PATH).remove("Gender")

In [18]:
print("Number of Ethenic",len(labelwe[labelwe==0]))
print("Number of western",len(labelwe[labelwe==1]))

Number of Ethenic 1715
Number of western 1873


In [19]:
print("western/ethenic data shape : ",westEth.shape," Label shape : ",labelwe.shape)

western/ethenic data shape :  (3588, 156, 156, 3)  Label shape :  (3588,)


In [20]:
testPercentage=0.3
x_train,x_test,y_train,y_test = train_test_split(westEth,labelwe,test_size = testPercentage,random_state=50, stratify=labelwe,shuffle=True)

In [21]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (2511, 156, 156, 3)  y_train shape :  (2511,)
x_test shape :  (1077, 156, 156, 3)  y_test shape :  (1077,)


In [22]:
print("Number of train ethenic",len(y_train[y_train==0]))
print("Number of train western",len(y_train[y_train==1]))

print("Number of test  ethenic",len(y_test[y_test==0]))
print("Number of test western",len(y_test[y_test==1]))

Number of train ethenic 1200
Number of train western 1311
Number of test  ethenic 515
Number of test western 562


In [23]:
NO_OF_EPOCHS=100
BATCH_SIZE=32

model2=Sequential()
model2.add(Conv2D(32,kernel_size=3, padding='same',activation='relu',input_shape=(156, 156,3)))
model2.add(MaxPool2D(pool_size=(2, 2)))
model2.add(Conv2D(64,kernel_size=3, padding='same',activation='relu'))
model2.add(MaxPool2D(pool_size=(2, 2)))
model2.add(Conv2D(128,kernel_size=3, padding='same',activation='relu'))
model2.add(Dropout(0.1))
model2.add(Flatten())
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1,activation='sigmoid'))
model2.summary()
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 156, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 78, 78, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 78, 78, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 39, 39, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 128)       73856     
                                                                 
 dropout (Dropout)           (None, 39, 39, 128)       0

In [ ]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=2, min_lr=0.001)
model2.fit(x_train,y_train,epochs=NO_OF_EPOCHS,verbose=1,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.7880 - accuracy: 0.5129
Epoch 1: val_accuracy improved from -inf to 0.58032, saving model to /content/drive/MyDrive/Results1/checkpoint-0001.hdf5
79/79 [==============================] - 105s 1s/step - loss: 0.7880 - accuracy: 0.5129 - val_loss: 0.6792 - val_accuracy: 0.5803 - lr: 0.0010
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 0.6579 - accuracy: 0.6149
Epoch 2: val_accuracy improved from 0.58032 to 0.64624, saving model to /content/drive/MyDrive/Results1/checkpoint-0002.hdf5
79/79 [==============================] - 105s 1s/step - loss: 0.6579 - accuracy: 0.6149 - val_loss: 0.6477 - val_accuracy: 0.6462 - lr: 0.0010
Epoch 3/100
79/79 [==============================] - ETA: 0s - loss: 0.5992 - accuracy: 0.6766
Epoch 3: val_accuracy did not improve from 0.64624
79/79 [==============================] - 106s 1s/step - loss: 0.5992 - accuracy: 0.6766 - val_loss: 0.6599 - val_accuracy: 0.6305 -

In [ ]:
model2.load_weights(resultPath + "/checkpoint-0015.hdf5")

In [ ]:
prediction_prob1 = model2.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

34/34 [==============================] - 12s 327ms/step


In [ ]:
m1model = Model(inputs=model2.input,outputs=model2.get_layer('dense_1').output)

In [ ]:
m1_x_train = m1model.predict(x_train,verbose=1)
m1_x_test = m1model.predict(x_test,verbose=1)

34/34 [==============================] - 10s 304ms/step


In [ ]:
print("Shape of model1 Train and Test DF : ",m1_x_train.shape," : ",m1_x_test.shape)

Shape of model1 Train and Test DF :  (2511, 1)  :  (1077, 1)


In [ ]:
def printMetrics(true,pred):
    print("Accuracy : ",accuracy_score(true, pred))
    print("Precision",precision_score(true, pred , average="weighted"))
    print("Recall : ",recall_score(true, pred , average="weighted"))
    print("F1-score : ",f1_score(true, pred, average="weighted"))
    print("Confusion Matrix : ")
    print(confusion_matrix(true, pred))
    print(classification_report(true,pred))

In [ ]:
RSVM = svm.SVC(kernel='rbf',probability=True)
RSVM.fit(m1_x_train, y_train)
RSVMprob = RSVM.predict_proba(m1_x_test)
y_pred = RSVM.predict(m1_x_test)
print("cost-insensitive")
printMetrics(y_test,y_pred)

cost-insensitive
Accuracy :  0.6833797585886723
Precision 0.6831189081190299
Recall :  0.6833797585886723
F1-score :  0.6830361699251003
Confusion Matrix : 
[[335 180]
 [161 401]]
              precision    recall  f1-score   support

           0       0.68      0.65      0.66       515
           1       0.69      0.71      0.70       562

    accuracy                           0.68      1077
   macro avg       0.68      0.68      0.68      1077
weighted avg       0.68      0.68      0.68      1077



Gender Classification

In [8]:
DATA_PATH = '/content/drive/MyDrive/MlProject1/ML_project1/Gender/'
genderData=[]
glabel=[]
ghashList=[]
count=2
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count-=1
    print(count)
    
    for file in os.listdir(DATA_PATH+folder):
         if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in ghashList):
              genderData.append(resize(img, (156, 156, 3)))
              ghashList.append(hsh)
              glabel.append(count)
genderData=np.array(genderData)
glabel=np.array(glabel)

>>>Reading  male
1
>>>Reading  female
0


In [10]:
resultPath = '/content/drive/MyDrive/MlProject1/gResult'
train_folder = os.listdir(DATA_PATH)

In [11]:
print("Number of maleGarments",len(glabel[glabel==0]))
print("Number of femaleGarments",len(glabel[glabel==1]))

Number of maleGarments 1825
Number of femaleGarments 1797


In [12]:
print("Gender data shape : ",genderData.shape," Label shape : ",glabel.shape)

Gender data shape :  (3622, 156, 156, 3)  Label shape :  (3622,)


In [13]:
testPercentage=0.3
x_train,x_test,y_train,y_test = train_test_split(genderData,glabel,test_size = testPercentage,random_state=50, stratify=glabel,shuffle=True)

In [14]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (2535, 156, 156, 3)  y_train shape :  (2535,)
x_test shape :  (1087, 156, 156, 3)  y_test shape :  (1087,)


In [15]:
print("Number of train male male",len(y_train[y_train==0]))
print("Number of train female garments",len(y_train[y_train==1]))

print("Number of test male garments ",len(y_test[y_test==0]))
print("Number of test female garments",len(y_test[y_test==1]))

Number of train male male 1277
Number of train female garments 1258
Number of test male garments  548
Number of test female garments 539


In [24]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=2, min_lr=0.001)
model2.fit(x_train,y_train,epochs=NO_OF_EPOCHS,verbose=1,batch_size=BATCH_SIZE,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.9594 - accuracy: 0.5424
Epoch 1: val_accuracy improved from -inf to 0.55246, saving model to /content/drive/MyDrive/Results1/checkpoint-0001.hdf5
79/79 [==============================] - 150s 2s/step - loss: 0.9594 - accuracy: 0.5424 - val_loss: 0.6755 - val_accuracy: 0.5525 - lr: 0.0010
Epoch 2/100
79/79 [==============================] - ETA: 0s - loss: 0.6586 - accuracy: 0.6181
Epoch 2: val_accuracy improved from 0.55246 to 0.59796, saving model to /content/drive/MyDrive/Results1/checkpoint-0002.hdf5
79/79 [==============================] - 150s 2s/step - loss: 0.6586 - accuracy: 0.6181 - val_loss: 0.6676 - val_accuracy: 0.5980 - lr: 0.0010
Epoch 3/100
79/79 [==============================] - ETA: 0s - loss: 0.5976 - accuracy: 0.6802
Epoch 3: val_accuracy improved from 0.59796 to 0.64253, saving model to /content/drive/MyDrive/Results1/checkpoint-0003.hdf5
79/79 [==============================] - 152s 2s/step - l

In [25]:
model2.load_weights(resultPath + "/checkpoint-0086.hdf5")

In [26]:
prediction_prob1 = model2.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

34/34 [==============================] - 26s 711ms/step


In [27]:
m1model = Model(inputs=model2.input,outputs=model2.get_layer('dense_1').output)

In [28]:
m1_x_train = m1model.predict(x_train,verbose=1)
m1_x_test = m1model.predict(x_test,verbose=1)

34/34 [==============================] - 13s 389ms/step


In [29]:
print("Shape of model1 Train and Test DF : ",m1_x_train.shape," : ",m1_x_test.shape)

Shape of model1 Train and Test DF :  (2511, 1)  :  (1077, 1)


In [30]:
def printMetrics(true,pred):
    print("Accuracy : ",accuracy_score(true, pred))
    print("Precision",precision_score(true, pred , average="weighted"))
    print("Recall : ",recall_score(true, pred , average="weighted"))
    print("F1-score : ",f1_score(true, pred, average="weighted"))
    print("Confusion Matrix : ")
    print(confusion_matrix(true, pred))
    print(classification_report(true,pred))

In [31]:
RSVM = svm.SVC(kernel='rbf',probability=True)
RSVM.fit(m1_x_train, y_train)
RSVMprob = RSVM.predict_proba(m1_x_test)
y_pred = RSVM.predict(m1_x_test)
print("cost-insensitive")
printMetrics(y_test,y_pred)

cost-insensitive
Accuracy :  0.6731662024141133
Precision 0.6729064913346853
Recall :  0.6731662024141133
F1-score :  0.6725119683825506
Confusion Matrix : 
[[324 191]
 [161 401]]
              precision    recall  f1-score   support

           0       0.67      0.63      0.65       515
           1       0.68      0.71      0.69       562

    accuracy                           0.67      1077
   macro avg       0.67      0.67      0.67      1077
weighted avg       0.67      0.67      0.67      1077

